In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt



2024-12-10 11:12:38.722615: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-10 11:12:38.722933: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 11:12:38.724713: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 11:12:38.729477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733825558.737615   86449 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733825558.74

In [ ]:
import os
import shutil
import pathlib

# Ursprünglicher Datenordner
original_dir = pathlib.Path("Data")  # Oberordner der Kategorien
new_base_dir = pathlib.Path("cats_vs_dogs_small")  # Zielbasisordner

def make_subset(subset_name, start_index, end_index):
    # Kategorien (Ordner: Cat und Dog)
    for category in ("Cat", "Dog"):
        # Neues Zielverzeichnis erstellen
        dir = new_base_dir / subset_name / category.lower()  # Kategorienamen in Kleinbuchstaben
        os.makedirs(dir, exist_ok=True)  # Ordner erstellen, falls nicht vorhanden

        # Bilder dieser Kategorie auswählen
        fnames = [f"{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            # Quell- und Zielpfade definieren
            src = original_dir / category / fname
            dst = dir / fname

            # Datei kopieren, wenn sie existiert
            if src.exists():
                shutil.copyfile(src, dst)

# Teilmengen erstellen
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)


## Deep Learning with Python Second Edititon (2021)
---

Kapitel 8: Introduction to deep learning for computer vision

*S. 202* 

Listing 8.1 Instantiating a small convnet



In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

*Seite 204*

Listing 8.3 Training the convnet on MNIST images

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)


S.204 

Listing 8.4 Evaluating the convnet

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

*S. 210*


Listing 8.5 An incorrectly structured convnet missing its max-pooling layers

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model_no_max_pool.summary()

# Dog vs Cats
---

Beispiel Modelle um ein Covnet aufzubauen
---
*S.216*

Listing 8.7 Instantiating a small convnet for dogs vs. cats classification

#### 1 Model: convnet_from_scratch

In [3]:
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

W0000 00:00:1733825567.902531   86449 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        12,545 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 991,041 (3.78 MB)

 Trainable params: 991,041 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

Listing 8.8 Configuring the model for training

In [5]:
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

##### image_dataset_from_directory: 
Ein Aufruf von 
- *image_dataset_from_directory(main_directory, labels='inferred')* 

gibt einen *tf.data.Dataset* zurück, der Stapel von Bildern aus den Unterverzeichnissen *class_a* und *class_b* zusammen mit den Labels 0 und 1 liefert (0 entspricht class_a und 1 entspricht class_b).

- Supported image formats: .jpeg, .jpg, .png, .bmp, .gif. Animated gifs are truncated to the first frame.

Paras:
1. directory: 
- Verzeichnis, in dem sich die Daten befinden. Wenn labels „inferred“ (abgeleitet) ist, sollte es Unterverzeichnisse enthalten, die jeweils Bilder für eine Klasse enthalten. Andernfalls wird die Verzeichnisstruktur ignoriert.

2. labels:
- Entweder „inferred“ (Bezeichnungen werden aus der Verzeichnisstruktur generiert), None (keine Bezeichnungen) oder eine Liste/ein Tupel von ganzzahligen Bezeichnungen, die so groß sind wie die Anzahl der im Verzeichnis gefundenen Bilddateien. Die Bezeichnungen sollten nach der alphanumerischen Reihenfolge der Pfade der Bilddateien sortiert sein (in Python über os.walk(directory) ermittelt).
3. image_size:    
- image_size=(180, 180),  Größe der Bilder, auf die sie beim Laden skaliert werden

4. batch_size:
batch_size=32
Anzahl der Bilder pro Batch



- base_dir / "train", base_dir / "validation", base_dir / "test": Diese Verzeichnisse enthalten die Bilddaten, die für das Training, die Validierung und das Testen des Modells verwendet werden.
- image_size=(180, 180): Jedes Bild wird auf eine Größe von 180x180 Pixeln skaliert, da CNNs eine feste Eingabegröße benötigen. Dies ist die Größe, die das Modell erwartet.
- batch_size=32: Die Daten werden in Batches von je 32 Bildern verarbeitet. Das bedeutet, dass beim Training in jedem Schritt 32 Bilder gleichzeitig verarbeitet werden.
- train_dataset, validation_dataset, test_dataset: Diese Objekte sind TensorFlow Dataset-Objekte, die die entsprechenden Bilddaten und Labels enthalten. Sie sind so strukturiert, dass sie effizient durch die Daten iterieren können, um sie an das Modell zu übergeben.

---
*S. 217*

Listing 8.9 Using image_dataset_from_directory to read images

In [6]:

train_dataset = image_dataset_from_directory(
    "cats_vs_dogs_small/train",
    image_size=(180, 180),
    batch_size=32)
    
validation_dataset = image_dataset_from_directory(
    "cats_vs_dogs_small/validation",
    image_size=(180, 180),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    "cats_vs_dogs_small/test",
    image_size=(180, 180),
    batch_size=32)

Found 1987 files belonging to 2 classes.
Found 993 files belonging to 2 classes.
Found 1987 files belonging to 2 classes.


- for data_batch, labels_batch in train_dataset:: Diese Schleife iteriert durch den train_dataset, der aus Batches von Bildern und den zugehörigen Labels besteht.

    - In jedem Schritt der Schleife werden ein data_batch (Bilder) und ein labels_batch (Labels) geladen und zur Verfügung gestellt.

- data_batch.shape: Dies gibt die Form des data_batch-Arrays aus. Da die Batch-Größe 32 ist, werden die Daten in einem 4D-Tensor organisiert:

    - (32, 180, 180, 3) bedeutet:
        - 32: Die Batch-Größe (32 Bilder).
        - 180, 180: Die Höhe und Breite der Bilder (180x180 Pixel).
        - 3: Die Anzahl der Farbkanäle (RGB).

- labels_batch.shape: Dies gibt die Form des labels_batch-Arrays aus. Da es sich bei der Klassifikation wahrscheinlich um eine Multiklassen-Aufgabe handelt, ist die Form (32,), was bedeutet, dass jedes der 32 Bilder mit einem entsprechenden Label verknüpft ist.

    - Wenn labels="inferred" verwendet wird, wird davon ausgegangen, dass das Verzeichnis so strukturiert ist, dass die Ordnernamen die Klassennamen oder Labels sind. Zum Beispiel könnte ein Ordner cat für Bilder der Klasse "Katze" und ein Ordner dog für Bilder der Klasse "Hund" stehen.

- break: Der break-Befehl unterbricht die Schleife nach dem ersten Batch. Das bedeutet, dass nur das erste Batch von 32 Bildern und Labels verarbeitet wird und anschließend der Code gestoppt wird. Normalerweise würde die Schleife ohne break durch alle Batches des Datensatzes iterieren.

---
*S. 219*

Listing 8.10 Displaying the shapes of the data and labels yielded by the Dataset

In [7]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break


data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


Callbacks ModelCheckpoint-Callback der dafür sorgt, dass das MOdell während des trainings gespeichert wird, aber nur unter bestimmten bediungen.

1. keras.callbacks.ModelCheckpoint
- ModelCheckpoint ist ein Callback in Keras, das während des Trainings eingesetzt wird, um das Modell nach jedem Trainigsepoch zu speichern. Dieser Callback kann so konfiguriert werden, dass das Modell nur dann gespeichert wird, wenn es eine bestimmte Bedingung erfüllt wird. (z.b wenn sich das Modell verbesser)
- Der Callback wird an die Methode *fit()* des Modells übergeben, und zwar als Teil der Liste von Callbacks, die das Modell während des Trainings anwendet. Das Bedeutet, dass bei jedem Epochenergebnis überprüft wird, ob das Modell gespeichert werden soll.

2. Parameter des ModelCheckpoint-Callbacks
- filepath= "convnet_from_scratch.keras":
    - Dateipfad wo das Modell gespeichert werden soll. Diese Datei enthält **alle** Modellgewichte sowie die Modellarchitektur und kann später für das Laden des Modells verwendet werden.

    - Der Dateiname kann auch die Epochenummer beinhalten "model_epoch_{epoch}.keras"
- save_best_only=True
    - Modelle werden nur gespeichert wenn sich das Modell verbessert. Nur das beste Modell basierend auf dem beobachteten Kriterium wird gespeichert.
    Auf *false* gesetzt wird wprde das Modell nach jeder Epoche gespeichert werden unabhängig davon ob sich die Leistung verbessert hat oder nicht.

- monitor ="val_loss"
    - Überwachung der Metrik die während des Training überwacht werden soll. In unseren Fall die Validierungsdaten *(val_loss)*
    - Das Modell wird nur gespeichert, wenn der val_loss in einer Epoche besser wird als der zuvor beste Wert.
    - Sie können auch andere Metriken überwachen, wie z. B. accuracy oder val_accuracy, je nachdem, welche Metrik für Ihr Modell wichtig ist.


Speicherung des Models

*S. 219*

Listing 8.11 Fitting the model using a Dataset

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="model/convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]

In [9]:
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/5
62/63 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.4888 - loss: 0.7034

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 125ms/step - accuracy: 0.4893 - loss: 0.7031 - val_accuracy: 0.5025 - val_loss: 0.6921
Epoch 2/5
62/63 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.5607 - loss: 0.6918

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.5609 - loss: 0.6917 - val_accuracy: 0.6224 - val_loss: 0.6601
Epoch 3/5
62/63 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.6095 - loss: 0.6664

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.6096 - loss: 0.6663 - val_accuracy: 0.6485 - val_loss: 0.6388
Epoch 4/5
62/63 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.6451 - loss: 0.6288

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 122ms/step - accuracy: 0.6454 - loss: 0.6287 - val_accuracy: 0.5418 - val_loss: 1.0411
Epoch 5/5
62/63 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.6694 - loss: 0.6401

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.6700 - loss: 0.6388 - val_accuracy: 0.6415 - val_loss: 0.7041


In [ ]:
import os
from PIL import Image

def check_images_in_directory(directory_path):
    # Alle Dateien im Verzeichnis auflisten
    filenames = os.listdir(directory_path)
    
    for filename in filenames:
        # Nur .jpg Bilder verarbeiten
        if filename.lower().endswith('.jpg'):
            filepath = os.path.join(directory_path, filename)
            try:
                # Bild öffnen und verifizieren
                img = Image.open(filepath)
                img.verify()  # Überprüft, ob die Datei ein gültiges Bild ist
                
                print(f"Bild ist gültig: {filename}")
            except (IOError, SyntaxError) as e:
                print(f"Ungültiges Bild: {filename} - Fehler: {e}")

# Verzeichnis der Bilder
path_images = '/home/riccardodandrea/Schreibtisch/Github/SceneDetect/cats_vs_dogs_small/test/dog'

# Alle Bilder im Verzeichnis überprüfen
check_images_in_directory(path_images)


Bild ist gültig: 1500.jpg
Bild ist gültig: 1501.jpg
Bild ist gültig: 1502.jpg
Bild ist gültig: 1503.jpg
Bild ist gültig: 1504.jpg
Bild ist gültig: 1505.jpg
Bild ist gültig: 1506.jpg
Bild ist gültig: 1507.jpg
Bild ist gültig: 1508.jpg
Bild ist gültig: 1509.jpg
Bild ist gültig: 1510.jpg
Bild ist gültig: 1511.jpg
Bild ist gültig: 1512.jpg
Bild ist gültig: 1513.jpg
Bild ist gültig: 1514.jpg
Bild ist gültig: 1515.jpg
Bild ist gültig: 1516.jpg
Bild ist gültig: 1517.jpg
Bild ist gültig: 1518.jpg
Bild ist gültig: 1519.jpg
Bild ist gültig: 1520.jpg
Bild ist gültig: 1521.jpg
Bild ist gültig: 1522.jpg
Bild ist gültig: 1523.jpg
Bild ist gültig: 1524.jpg
Bild ist gültig: 1525.jpg
Bild ist gültig: 1526.jpg
Bild ist gültig: 1527.jpg
Bild ist gültig: 1528.jpg
Bild ist gültig: 1529.jpg
Bild ist gültig: 1530.jpg
Bild ist gültig: 1531.jpg
Bild ist gültig: 1532.jpg
Bild ist gültig: 1533.jpg
Bild ist gültig: 1534.jpg
Bild ist gültig: 1535.jpg
Bild ist gültig: 1536.jpg
Bild ist gültig: 1537.jpg
Bild ist gül

*S. 220*

Listing 8.12 Displaying curves of loss and accuracy during training

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

# Plotting
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

*S. 221* 

Listing 8.13 Evaluating the model on the test set

In [ ]:
test_model = keras.models.load_model("model/convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

##### **Test accuracy von 0.693**

Der **data_augmentation-Block** erstellt eine Pipeline für die Bildaugmentation, bei der:

Zufällige horizontale Spiegelungen durchgeführt werden.
Zufällige Rotationen von Bildern im Bereich von -10 bis +10 Grad angewendet werden.
Zufälliger Zoom von bis zu 20% durchgeführt wird.

Das hilft, das Modell robuster gegen verschiedene Bildorientierungen zu machen und verhindert, dass das Modell nur spezifische Ausrichtungen von Objekten erkennt.

*S.221*
Listing 8.14 Define a data augmentation stage to add to an image model

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

*S. 222*

Listing 8.15 Displaying some randomly augmented training images

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1): #welches bild
    for i in range(9): # bilder anzahl
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1) # wie viele plots
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

binäre Klassifikationsaufgabe eines CNN Models:
- **Input 180 x 180 x 3 Farbkanäle RGB**
- **data_augmentation(inputs)** Transormierten bilder mit den bilder inputs gespiegelt, Rotationen und Zoom.
- **Reskalierung: (x = layers.Rescaling(1./255)(x))**
Die Reskalierung sorgt dafür, dass die Pixelwerte der Bilder von einem Bereich von [0, 255] (die typischen Farbwerte von Bildern) auf [0, 1] skaliert werden.
Dies ist wichtig, weil es den Lernprozess stabiler macht und das Modell schneller konvergieren kann, da viele neuronale Netzwerke besser mit kleineren Eingabewerten arbeiten.

- **Konvolutionale Schichten (Conv2D)**
    - Jede Conv2D-Schicht hat eine bestimmte Anzahl von Filtern (oder Kernels) und eine Kernelgröße (hier 3x3). Diese Filter werden über das Bild geschoben, um lokale Merkmale wie Kanten, Ecken oder Texturen zu extrahieren.
    -Die Anzahl der Filter wächst mit den Schichten (32, 64, 128, 256), was darauf hindeutet, dass das Modell zunehmend komplexere Merkmale erlernt, je weiter es in der Tiefe geht.
    - activation="relu" bedeutet, dass die ReLU-Aktivierungsfunktion auf die Ausgaben jeder Convolutional-Schicht angewendet wird. ReLU hilft, Nichtlinearitäten in das Modell einzuführen, wodurch es leistungsfähiger wird.

- Max-Pooling-Schichten (MaxPooling2D): Nach jeder Convolutional-Schicht folgt eine Max-Pooling-Schicht, die die Dimensionen der Feature-Maps reduziert.
- Der Parameter pool_size=2 bedeutet, dass der Pooling-Bereich 2x2 ist. Max-Pooling extrahiert das Maximum aus einem 2x2 Bereich der Feature-Map, was zu einer Reduzierung der Größe und einer Verdichtung der Informationen führt.
- Dies hilft, die Rechenleistung zu reduzieren und das Modell robuster gegenüber kleinen Verschiebungen im Bild zu machen.

- **Flatten**
- Flatten() wird verwendet, um die mehrdimensionalen Ausgaben der letzten Konvolutional- und Pooling-Schicht in einen flachen Vektor umzuwandeln.
- Dies ist notwendig, da die nachfolgende dichte Schicht (Dense) nur mit Vektoren arbeitet (nicht mit mehrdimensionalen Tensoren).

Dropout-Sicht (Dropout=0.50)
- Regulierungstechnik um Overfitting zu verhindern.
- Während des Trainings zufällig 50 % der Neuronen in dieser Sicht deaktivert (auf Null gesetzt)
- Dies zwingt das Modell, robustere Merkmale zu lernen, da es nicht auf bestimmte Neuronen angewiesen ist.

- Ausgabeschicht (Dense)
- Dense(1, activation="sigmoid") definiert eine dichte Schicht mit nur einem Neuron und der Sigmoid-Aktivierungsfunktion.
- Da es sich um ein binäres Klassifikationsproblem handelt, gibt die 
    - Sigmoid-Funktion eine Wahrscheinlichkeit zwischen 0 und 1 zurück, die angibt, ob das Bild der Klasse 1 oder Klasse 0 zugeordnet werden soll.


In [ ]:
import tensorflow as tf

# Modell laden
model_path = r"model\convnet_from_scratch.keras"
convnet_from_scratch = tf.keras.models.load_model(model_path)

# Modell Zusammenfassung anzeigen
convnet_from_scratch.summary()

In [ ]:
# Model nutzen
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Bild laden
image_path = r"cats_and_dogs_small\test\cats\1547.jpg"

# Bild auf die gewünschte Eingabegröße skalieren unsere Modell nimmt 180 x 180 x3
img = image.load_img(image_path, target_size=(180, 180))

# Bild anzeigen
plt.imshow(img)
plt.axis("off")
plt.show()


In [ ]:
# Das Bild in ein NumPy-Array umwandeln
img_array = image.img_to_array(img)
# Das Bild auf die Batch-Dimension erweitern (Modell erwartet eine Batch von Bildern)
img_array = np.expand_dims(img_array, axis=0)
print(img_array.shape)


In [ ]:
# Vorhersage mit dem geladenen Modell
prediction = convnet_from_scratch.predict(img_array)

# Vorhersage auswerten
# Da es eine binäre Klassifikation (Hund vs. Katze) ist:
if prediction[0] > 0.5:
    print("Das Bild zeigt einen Hund.")
else:
    print("Das Bild zeigt eine Katze.")

# Ausgabe der Vorhersagewahrscheinlichkeit
print("Vorhersagewahrscheinlichkeit für Hund:", prediction)


---
*S.223*

Listing 8.16 Defining a new convnet that includes image augmentation and dropout

#### 2 Model: convnet_from_scratch_with_augmentation

Modell erstellen

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

*S. 223*

Listing 8.17 Training the regularized convnet

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath="model/convnet_from_scratch_with_augmentation.keras",
    save_best_only=True,
    monitor="val_loss")
]

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

*S.224*

Visualisieren und Evaluieren

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

# Plotting
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

plt.show()

*S. 224*

Listing 8.18 Evaluating the model on the test set





In [ ]:
test_model = keras.models.load_model(
"model/convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

"convnet_from_scratch_with_augmentation" **Test accuracy: 0.795**

---

Model einladen: **"convnet_from_scratch_with_augmentation"** und 

##### Vohersagen

In [ ]:
import tensorflow as tf

# Modell laden
model_path = r"model\convnet_from_scratch_with_augmentation.keras"
model_convnet_from_scratch_with_augmentation = tf.keras.models.load_model(model_path)

# Modell Zusammenfassung anzeigen
model_convnet_from_scratch_with_augmentation.summary()

In [ ]:
# Model nutzen
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Bild laden
image_path = r"cats_and_dogs_small\test\cats\1500.jpg"

# Bild auf die gewünschte Eingabegröße skalieren unsere Modell nimmt 180 x 180 x3
img = image.load_img(image_path, target_size=(180, 180))

# Bild anzeigen (optional)
plt.imshow(img)
plt.axis("off")
plt.show()


In [ ]:
# Das Bild in ein NumPy-Array umwandeln
img_array = image.img_to_array(img)
# Das Bild auf die Batch-Dimension erweitern (Modell erwartet eine Batch von Bildern)
img_array = np.expand_dims(img_array, axis=0)
print(img_array.shape)


In [ ]:
# Vorhersage mit dem geladenen Modell
prediction = model_convnet_from_scratch_with_augmentation.predict(img_array)

# Vorhersage auswerten
# Da es eine binäre Klassifikation (Hund vs. Katze) ist:
if prediction[0] > 0.5:
    print("Das Bild zeigt einen Hund.")
else:
    print("Das Bild zeigt eine Katze.")

# Ausgabe der Vorhersagewahrscheinlichkeit
print("Vorhersagewahrscheinlichkeit für Hund:", prediction[0])


#### 3. Bereits erstelltest Model VGG16 nutzen:
---

Feature extraction consists of using the representations learned by a previously
trained model to extract interesting features from new samples. These features are
then run through a new classifier, which is trained from scratch.
As you saw previously, convnets used for image classification comprise two parts:
they start with a series of pooling and convolution layers, and they end with a densely
connected classifier. The first part is called the convolutional base of the model. In the
case of convnets, feature extraction consists of taking the convolutional base of a previously
trained network, running the new data through it, and training a new classifier
on top of the output.

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
weights="imagenet",
include_top=False,
input_shape=(180, 180, 3))

In [ ]:
conv_base.summary()

In [ ]:
import numpy as np
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)
train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

In [ ]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="model/feature_extraction.keras",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf

# Modell laden
model_path = r"model\feature_extraction.keras"
feature_extraction = tf.keras.models.load_model(model_path)

# Modell Zusammenfassung anzeigen
feature_extraction.summary()

#### MobileNetV2 mit den weights von imagenet

In [ ]:
import imageio.v2 as imageio
from PIL import Image
from keras.applications.mobilenet_v2 import MobileNetV2
import numpy as np
import imageio.v2 as imageio  
from keras.applications.mobilenet_v2 import preprocess_input
from keras.applications.mobilenet_v2 import decode_predictions
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
MobileNetV2 = MobileNetV2(weights='imagenet')
MobileNetV2

In [ ]:
# Bild laden
image_path = r"shi_zu.jpg"

# Bild auf die gewünschte Eingabegröße skalieren unsere Modell nimmt 180 x 180 x3
img = image.load_img(image_path, target_size=(224, 224))

# Bild anzeigen
plt.imshow(img)
plt.axis("off")
plt.show()


In [ ]:
img = imageio.imread(r'shi_zu.jpg')

# Bild auf die gewünschte Größe (224, 224) skalieren
img_resized = Image.fromarray(img).resize((224, 224))

# Bild in das numpy Array einfügen
data = np.empty((1, 224, 224, 3))
data[0] = np.array(img_resized)

In [ ]:
data = preprocess_input(data)
data.shape

In [ ]:
predictions = MobileNetV2.predict(data)
print('Shape: {}'.format(predictions.shape))

In [ ]:
output_neuron = np.argmax(predictions[0])
print('Most active neuron: {} ({:.2f}%)'.format(
    output_neuron,
    100 * predictions[0][output_neuron]
))


In [ ]:
for name, desc, score in decode_predictions(predictions)[0]:
    print('- {} ({:.2f}%%)'.format(desc, 100 * score))

---
##### Fine tune MobileNetV2

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.25)

---

### SENET 
Code: https://pytorch.org/hub/pytorch_vision_squeezenet/

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
# or
# model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1', pretrained=True)
#model.eval()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
image_path = r"cats_and_dogs_small\test\dogs\1586.jpg"

# Bild auf die gewünschte Eingabegröße skalieren unsere Modell nimmt 180 x 180 x3
img = Image.open(r"cats_and_dogs_small\test\dogs\1586.jpg")

# Bild anzeigen
plt.imshow(img)
plt.axis("off")
plt.show()


#### From dir path

In [ ]:
from PIL import Image
image = Image.open(r"cats_and_dogs_small\test\dogs\1586.jpg")


In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms

input_image = Image.open(r"cats_and_dogs_small\test\dogs\1586.jpg")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0].shape)
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities.shape)

In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

---

#### From Url example Code

In [ ]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0].shape)
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities.shape)

In [ ]:
# Read the categories

with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Fine tune SENET

In [ ]:
import torch
import torch.nn as nn

# Lade das vortrainierte SqueezeNet-Modell
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

# Behalte das Basis-Modell bei
base_model = model.features  # Das ist die 'Feature'-Extraktionseinheit

# Füge neue Schichten für die Klassifikation hinzu
num_classes = 5  # Anzahl der Klassen in deinem Datensatz

# Neue Klassifikationsschicht
new_classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),  # Reduziert die Ausgabedimensionen auf 1x1
    nn.Flatten(),  # Wandelt die 2D-Ausgabe in einen 1D-Vektor um
    nn.Linear(512, num_classes)  # Die letzte Schicht für die Klassifikation
)

# Kombiniere das Basis-Modell mit der neuen Klassifikationsschicht
model = nn.Sequential(
    base_model,
    new_classifier
)

# Initialisiere das Modell für das Training
model.train()

# Definiere den Optimierer und die Verlustfunktion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Beispiel für einen Trainingsloop
num_epochs = 5
for epoch in range(num_epochs):
    for inputs, labels in dataloader:  # Angenommen, du hast einen DataLoader `dataloader`
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


### Transformer:

In [ ]:
# Transformers ViT

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image

# Pfad zum Bild
image_path = r'cats_and_dogs_small\test\dogs\1515.jpg'  # Gib den vollständigen Pfad zum Bild an
image = Image.open(image_path)

# Processor und Modell laden
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

# Bild verarbeiten
inputs = processor(images=image, return_tensors="pt")

# Vorhersage machen
outputs = model(**inputs)
logits = outputs.logits

# Die vorhergesagte Klasse bestimmen
predicted_class_idx = logits.argmax(-1).item()

# Vorhersage ausgeben
print("Predicted class:", model.config.id2label[predicted_class_idx])
